In [1]:
from bs4 import BeautifulSoup
import urllib.request
import os
import requests
from selenium import webdriver
import datetime
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# 예보에서 가져와야 하는것

- 기온
- 강수형태(?)
- 강수확률(%)
- 습도(%)
- 풍속(m/s)
- 풍향(deg)
- 하늘상태
- 강수량(mm)
- 적설

## 시간별 
- 날씨(?), 강수확률(%), 강수량(mm), 적설(cm), 기온, 풍향, 풍속(m/s), 습도(%)

- 남은것 : 강수형태, 하늘상태
- 날씨가 무얼 의미하는지 찾아서 채우기 -> 그냥 그래픽에 같은 이름을 가진 요소가 있기에 그대로 가져다 쓰겠음

## 그래픽
- 강수형태, 하늘상태

# 시간별

In [2]:
column_names = ['date(current)', 
                'datetime(current)', 
                'date(prediction)', 
                'datetime(prediction)',
               '기온(℃)',
               '강수형태',
               '강수확률(%)',
               '습도(%)',
               '풍속(m/s)',
               '풍향(deg)',
               '하늘상태',
               '강수량(mm)',
               '적설(cm)']
df = pd.DataFrame(columns=column_names)
display(df)

,date(current),datetime(current),date(prediction),datetime(prediction),기온(℃),강수형태,강수확률(%),습도(%),풍속(m/s),풍향(deg),하늘상태,강수량(mm),적설(cm)


In [3]:
url_1 = 'https://web.kma.go.kr/weather/forecast/timeseries.jsp'

In [ ]:
#path_browser = os.path.abspath(os.path.join(os.getcwd(), 'chromedriver.exe')) 
browser = webdriver.Chrome()
browser.implicitly_wait(3)
browser.get(url_1)

In [ ]:
browser.execute_script('visibleDiv("layor_area",true, null, this); return false;')
browser.execute_script('selectLocal("CITY","4400000000","충청남도"); return false;')
browser.execute_script('selectLocal("DONG","4427000000","당진시"); return false;')
browser.execute_script('setLocal("4427032000","석문면"); return false;')
browser.execute_script('refresh_dfs(); return false;')
browser.execute_script('visibleDiv("layor_area",false); reloadTown(true);  return false;')

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

## date(current), datetime(current)

In [ ]:
datetime_current = soup.find('div', attrs={'class' : 'timeseries_subn'})
datetime_current = datetime_current.find('span').get_text()
print(datetime_current)
date_current = datetime_current[0:4] + '-' + datetime_current[6:8] + '-' + datetime_current[10:12] 
datetime_current = date_current + ' ' + datetime_current[20:25]+':00'
print(date_current)
print(datetime_current)

In [ ]:
table = soup.find('div', attrs={'class' : 'timeseries_mar'})
table = table.find('tbody')

## date(prediction), datetime(prediction)

In [ ]:
table_time = table.find('tr', attrs={'class' : 'time'})
table_time_all = table_time.findAll('p', attrs={'class' : 'time_hr'})
day = 0
time_last = 0

date_prediction_series = []
datetime_prediction_series = []

for idx, time in enumerate(table_time_all) :
    # initial setting
    datetime_prediction = ''
    if idx == 0 :
        time_last = int(time.get_text())
    
    # day changed?
    if int(time.get_text()) < time_last :
        day += 1
    
    # today, tomorrow, day after tomorrow
    if day == 0 :
        date_prediction = date_current
        datetime_prediction = date_prediction + ' ' + time.get_text() + ':00:00'
        date_prediction_series.append(date_prediction)
        datetime_prediction_series.append(datetime_prediction)
    if day == 1 :
        year = int(date_current[0:4])
        month = int(date_current[5:7])
        date = int(date_current[8:10])
        date_prediction = datetime.date(year, month, date)+datetime.timedelta(days=1)
        datetime_prediction = str(date_prediction) + ' ' + time.get_text() + ':00:00'
        date_prediction_series.append(date_prediction)
        datetime_prediction_series.append(datetime_prediction)
    if day == 2 :
        year = int(date_current[0:4])
        month = int(date_current[5:7])
        date = int(date_current[8:10])
        date_prediction = datetime.date(year, month, date)+datetime.timedelta(days=2)
        datetime_prediction = str(date_prediction) + ' ' + time.get_text() + ':00:00'
        date_prediction_series.append(date_prediction)
        datetime_prediction_series.append(datetime_prediction)
  
    time_last = int(time.get_text())
        
    print(datetime_prediction)

In [ ]:
df['datetime(prediction)'] = datetime_prediction_series
df['date(current)'] = date_current
df['datetime(current)'] = datetime_current
df['date(prediction)'] = date_prediction_series
display(df)

## ||Set date||

In [ ]:
#df = df.iloc[0:15, :]
display(df)

## 기온

In [ ]:
- 0 : 날짜
- 1 : 시각
- 2 : 날씨
- 3 : 강수확률(%)
- 4 : 강수량(mm), 적설(cm)
- 5 : 최저/최고(℃)
- 6 : 기온(℃)	
- 7 : 풍향 / 풍속(m/s)
- 8 : 습도(%)

In [ ]:
trs = table.findAll('tr')
for idx, tr in enumerate(trs) : 
    print(idx)
    print(tr)


- 2 : 날씨
- 눈, 흐림, 구름 많음, 구름 조금, 맑음

In [ ]:
weather = trs[2]
weather_series = []
for td in weather.findAll('td') :
    
    if td.get('title') != None :
        print(td.get("title"))
        weather_series.append(td.get('title'))

## 강수확률
- 3 : 강수확률(%)

In [ ]:
probs = trs[3]
prob_series = []
for prob in probs.findAll('td') :
    print(prob.get_text())
    if prob.get_text() != '' : 
        prob_series.append(prob.get_text())
print(len(prob_series))

In [ ]:
df['강수확률(%)'] = prob_series
display(df)

## 강수량, 적설
- 4 : 강수량(mm), 적설(cm)

In [ ]:
rainfalls_snows = trs[4]
rainfall_series = []
snow_series = []
first_size = 0  # 첫번째가 3시간이면 1, 첫번째가 6시간이면 2

for idx, rainfall_snow in enumerate(rainfalls_snows.findAll('td')) :
    try : 
        if idx == 0 :
            first_size = int(rainfall_snow.get('colspan'))
        
        temp = rainfall_snow
        try : # <br>이 있는 경우
            rainfall = rainfall_snow.br.previous_sibling.replace('\t', '').replace(' ', '').replace('\n', '')
            snow = rainfall_snow.br.next_sibling.replace('\t', '').replace(' ', '').replace('\n', '')
            if rainfall == '-' :
                rainfall = str(0)
            if snow == '-' :
                snow = str(0)
        except Exception as e : # <br>이 없는 경우 : 추측이지만 전부 - 일때인듯
            rainfall = str(0)
            snow = str(0)
            
        print(rainfall, snow)
        rainfall_series.append(rainfall)
        snow_series.append(snow)
    except Exception as e :
        print(e)
        print('td empty')
        
print(first_size)

- 첫번째 값이 3시간짜리인지, 6시간짜리인지 두가지 경우가 있음

In [ ]:
rainfall_series

In [ ]:
if first_size == 2 :
    for idx in range(len(rainfall_series)-1) :
        df['강수량(mm)'].iloc[idx*2] = rainfall_series[idx]
        df['적설(cm)'].iloc[idx*2] = snow_series[idx]
    
if first_size == 1 :
    df['강수량(mm)'].iloc[0] = rainfall_series[0]
    df['적설(cm)'].iloc[0] = snow_series[0]
    
    for idx in range(len(rainfall_series)-2) :
        df['강수량(mm)'].iloc[(idx*2)+1] = rainfall_series[idx]
        df['적설(cm)'].iloc[(idx*2)+1] = snow_series[idx]

df['강수량(mm)'] = df['강수량(mm)'].fillna(method='ffill')
df['적설(cm)'] = df['적설(cm)'].fillna(method='ffill')
display(df)

    
    

## 기온
- 6 : 기온

In [ ]:
celsious = trs[6]
celsious_series = []
for temperature in celsious.findAll('td') :
    celsious_series.append(temperature.get_text())
    print(temperature.get_text())

In [ ]:
print(len(celsious_series))
print(celsious_series)
celsious_series.pop()
print(len(celsious_series))
print(celsious_series)

In [ ]:
df['기온(℃)'] = celsious_series
display(df)

## 풍향/풍속

- 7 : 풍향/풍속


- 1 :  북
- 45 : 북동
- 90 : 동
- 135 : 남동
- 180 : 남
- 225 : 남서
- 270 : 서
- 315 : 북서

In [ ]:
wind_dir_vels = trs[7]
wind_dir_series = []
wind_vel_series = []
for wind_dir_vel in wind_dir_vels.findAll('td') :
    temp = wind_dir_vel.get('title')
    try : 
        wind_dir, wind_vel = temp.split(' ')
        if wind_dir.replace('풍', '') == '북' :
            wind_dir = 1
        elif wind_dir.replace('풍', '') == '북동' :
            wind_dir = 45
        elif wind_dir.replace('풍', '') == '동' :
            wind_dir = 90
        elif wind_dir.replace('풍', '') == '남동' :
            wind_dir = 135
        elif wind_dir.replace('풍', '') == '남' :
            wind_dir = 180
        elif wind_dir.replace('풍', '') == '남서' :
            wind_dir = 225
        elif wind_dir.replace('풍', '') == '서' :
            wind_dir = 270
        elif wind_dir.replace('풍', '') == '북서' :
            wind_dir = 315
        else :
            print('16방위가 생각한대로 나오지 않았다')
    except Exception as e :
        print(e)
        wind_dir = 'NaN'
        wind_vel = 'NaN'
    
        
        
    wind_dir_series.append(wind_dir)
    wind_vel_series.append(wind_vel.replace('m/s', ''))
    print(wind_dir, wind_vel.replace('m/s', ''))

In [ ]:
wind_dir_series.pop()
wind_vel_series.pop()

In [ ]:
df['풍속(m/s)'] = wind_vel_series
df['풍향(deg)'] = wind_dir_series
display(df)

## 습도


- 8 : 습도

In [ ]:
humidities = trs[8]
humidity_series = []
for humidity in humidities.findAll('td') :
    print(humidity.get_text())
    humidity_series.append(humidity.get_text())

In [ ]:
humidity_series.pop()

In [ ]:
df['습도(%)'] = humidity_series
display(df)

## 결과

In [ ]:
display(df)

## Manually modify

In [ ]:
display(df)

In [ ]:
#기온(℃)	
column_1 = [-1, -1, -1, 1, 2, 2, 1, 0, 1, 1, 2, 5, 6, 3, 2]
df['기온(℃)'] = column_1

In [ ]:
#강수형태	
column_2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
df['강수형태'] = column_2

In [ ]:
#강수확률(%)	
column_3 = []

In [ ]:
#습도(%)	
column_4 = [100, 100, 100, 100, 90, 90, 95, 100, 100, 95, 95, 70, 65, 75, 80]
df['습도(%)'] = column_4

In [ ]:
#풍속(m/s)	
column_5 = [5, 5, 5, 4, 3, 4, 6, 5, 5, 6, 6, 6, 4, 6, 7]
df['풍속(m/s)'] = column_5

In [ ]:
#풍향(deg)	
column_6 = [90, 90, 45, 1, 1, 315, 315, 315, 315, 315, 315, 315, 315, 270, 315]
df['풍향(deg)'] = column_6

In [ ]:
#하늘상태	
column_7 = [2, 2, 2,  2, 2, 2, 2, 2, 2, 2, 2,  3, 3, 3, 1, 1, 0, 0]
df['하늘상태'] = column_7

In [ ]:
#강수량(mm)	
#column_8 = [2.5, 2.5, 2.5, 2.5, 2.5, 0.5, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
#df['강수량(mm)'] = column_8
df['강수량(mm)'].iloc[1:3] = [2.5, 2.5]

In [ ]:
#적설(cm)
#column_9 = [2.5, 2.5, 2.5, 2.5, 2.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#df['적설(cm)'] = column_9
df['적설(cm)'].iloc[1:3] = [2.5, 2.5]

In [ ]:
display(df)

# Save as .csv file

In [ ]:
path = os.path.abspath(os.path.join(os.getcwd(), '5Mar1400.csv'))
print(path)
df.to_csv(path, sep='\t')

In [ ]:
df_read = pd.read_csv(path, sep='\t')
display(df_read)